In [ ]:
import os
import pickle
import secrets
import math

# Précalculer une table de petits nombres premiers
small_primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997]

def is_prime(num, k=10):
    if num <= 1:
        return False
    for p in small_primes:  # Test rapide avec les petits nombres premiers
        if num % p == 0:
            return False
    if num <= 3:
        return True
    if num % 2 == 0 or num % 3 == 0:
        return False

    d = num - 1
    while d % 2 == 0:
        d //= 2

    for _ in range(k):
        a = secrets.randbelow(num - 3) + 2
        x = pow(a, d, num)
        if x == 1 or x == num - 1:
            continue
        return False  # Pas besoin de la boucle interne optimisée car on utilise pow
    return True

def generate_prime(bits):
    while True:
        candidate = secrets.randbits(bits)
        # Tester la divisibilité par les petits nombres premiers
        for p in small_primes:
            if candidate % p == 0 and p**2 <= candidate:
                break
        else:
            if is_prime(candidate):
                return candidate


def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

def generate_keypair(p, q):
    if p == q:
        raise ValueError("p and q cannot be equal")
    if gcd(p-1, q-1) > 2**(256) :
        raise ValueError("gcd(p-1,q-1) est trop grand, risque d'attaque de Wiener")
    n = p * q
    phi = (p - 1) * (q - 1)
    e = 65537
    d = pow(e, -1, phi)
    return ((e, n), (d, n))

class PaddingError(Exception):
    pass

def pkcs1_pad(data, k):
    if len(data) > k - 11:
        raise ValueError("Message too long")
    padding_length = k - len(data) - 3
    padding = b'\x00\x02' + secrets.token_bytes(padding_length) + b'\x00'
    return padding + data

def pkcs1_unpad(data):
    if len(data) < 2 or data[0] != 0x00 or data[1] != 0x02:
        raise PaddingError("Invalid PKCS#1 padding: incorrect prefix")

    i = 2
    while i < len(data) and data[i] != 0x00:
        i += 1

    if i >= len(data) or i == 2:
        raise PaddingError("Invalid PKCS#1 padding: no separator byte found")

    return data[i + 1:]

def encrypt(message, public_key):
    e, n = public_key
    padded_message = pkcs1_pad(message, (n.bit_length() + 7) // 8)
    ciphertext = pow(int.from_bytes(padded_message, 'big'), e, n)
    return ciphertext

def decrypt(ciphertext, private_key):
    d, n = private_key
    padded_message = pow(ciphertext, d, n).to_bytes((n.bit_length() + 7) // 8, byteorder='big')
    try:
        return pkcs1_unpad(padded_message)
    except PaddingError as e:
        print(f"Decryption error: {e}")
        return None

def save_keys(public_key, private_key, filename_public, filename_private):
    with open(filename_public, 'wb') as f:
        pickle.dump(public_key, f)
    with open(filename_private, 'wb') as f:
        pickle.dump(private_key, f)

def load_keys(filename_public, filename_private):
    try:
        with open(filename_public, 'rb') as f:
            public_key = pickle.load(f)
        with open(filename_private, 'rb') as f:
            private_key = pickle.load(f)
        return public_key, private_key
    except FileNotFoundError:
        return None, None


In [ ]:
if __name__ == "__main__":
    # Vérification de l'existence des clés
    public_key, private_key = load_keys('public.key', 'private.key')
    if public_key is None or private_key is None:
        # Génération des clés si elles n'existent pas
        p = generate_prime(2048)
        q = generate_prime(2048)
        try:
            public_key, private_key = generate_keypair(p, q)
        except ValueError as e:
            print(f"Erreur lors de la génération des clés : {e}")
            exit()
        save_keys(public_key, private_key, 'public.key', 'private.key')
        print("Clés générées et sauvegardées.")
    else :
        print("Clés chargées depuis les fichiers.")


    # Saisie du message par l'utilisateur
    message = input("Entrez le message à chiffrer : ")
    message = message.encode('utf-8')

    # Chiffrement du message
    ciphertext = encrypt(message, public_key)
    if ciphertext is None:
        print("Le message n'a pas pu être chiffré (erreur de padding probablement).")
        exit()

   # Déchiffrement du message
decrypted_message = decrypt(ciphertext, private_key)

print("Message original:", message)
print("Message chiffré:", ciphertext)
if decrypted_message:
    try:
        print("Message déchiffré:", decrypted_message.decode('utf-8')) # Attempt decoding
    except UnicodeDecodeError:
        print("Message déchiffré (raw bytes):", decrypted_message) # Fallback to raw bytes
else:
    print("Le message n'a pas pu être déchiffré.")

Clés chargées depuis les fichiers.
Entrez le message à chiffrer : salut
Message original: b'salut'
Message chiffré: 25571525282170663984423893455976920093597131551160187068974064431948307848051059059787147965794395449977034042131571210787721216177486139426246854387167122152130874447664178725476342116871374074406065796985532752866567196531137118427644767910178728105911143608687798231672192660439047315810482857723356164943440793524237855232551691093237600040855412308862605082387876308304178266122806948170486370634217255234444193689409144559078718452257641593447640389192673353844330608074277392296960617749419539953444248354945502752730635632143678497596519720470159783907188694470552638582375433402225963585115365521931833982021069700999283118873945222511248763436674908937243955750832017059909809169934581383097059503581166158207231723877700045667791432323395719241394944936673844201155683675363303908011575774852392268184783309296373250133325864941768031616973530422555333441901739236484028926